Here, you develop the 

In [1]:
import os.path
from scipy import *
import numpy as np
from numpy import *
from numpy import linalg as LA
from scipy import linalg as LA2
import sympy as sympy
import sys as sys
import time
import matplotlib.pyplot as plt
import itertools as it
from IPython.core.display import HTML

sys.path.append('/Users/sashacurcic/SashasDirectory/ANAG/FV_MG/')
from Modules import BasicTools as BT
from Modules import WaveTools as WT
from Modules import PlotTools as PT
from Modules import FFTTools as FFTT
from Modules import OperatorTools as OT
from Modules import GridTransferTools as GTT
from Modules import TestTools as TT
from Modules import SolverTools as ST

display(HTML("<style>pre { white-space: pre !important; }</style>"))
np.set_printoptions( linewidth = 10000, threshold = 100000)

This cell takes all of the input parameters.

In [2]:
nh_min = 16
refRatio = 2
c = 1.
CFL = 0.5
nt = 32
RK = 4
deriv = 'CD'
order = 4

This cell contains the relevant switch information.

In [3]:
if (RK == 1):
    TimeIntegratorFunc = ST.ForwardEuler
else:
    if (RK == 2):
        TimeIntegratorFunc = ST.MidpointMeth
    else:
        TimeIntegratorFunc = ST.RK4

if (deriv == 'U'):
    DiffFunc = ST.Upwind
else:
    DiffFunc = TT.CenterDiff#ST.CenterDiff

This cell instantiates the grid, derivative operators, and Fourier modes.

In [4]:
omega = BT.Grid(nh_min)
finehalf = list(np.arange(int(nh_min / refRatio)))# + int(nh / refRatio))
omega.AddPatch(refRatio, finehalf)
degFreed = omega.degFreed
nh_max = omega.nh_max
t, nt = ST.CalcTime(omega, CFL, c, nt = nt)
derivMat = OT.SpaceDeriv(omega, order, deriv)
spatOp = -c * derivMat
strings = omega.strings
timePropOp = LA2.expm(t * spatOp)
omegaF = BT.Grid(nh_max)
wavesF = WT.MakeWaves(omegaF)

In [5]:
polyInterp = GTT.GhostCellStencil(order, -0.5)
print(polyInterp)
print(type(polyInterp))
print(order)

(array([-0.02857143,  0.57142857,  0.78571429, -0.41428571,  0.08571429]), 2, 3)
<class 'tuple'>
4


This cell runs a test on a fifth-order cell-averaged polynomial interpolation for a ghost cell at the edge of the coarse-fine interface. If it doesn't reach the appropriate order of accuracy, there is an `assert` which force quits.

In [6]:
print(order)
def TestPoly(func, args, const = 2, tol = 1e-10):
    errorLoc = 'Location:\n'
    errorMess = ''
    
    # Create stencil.
    
    out = func(*args)
    if (func == GTT.GhostCellStencil):
        
        polyInterp, n_c, n_f = out
        
        # Create vector of cell bounds.
        bounds, n_c, n_f = GTT.BoundVals(*args)
        order, x_0 = args
        
        if (x_0 > 0):
            x_1 = x_0 - 0.5
            x_2 = x_0
        else:
            if (x_0 < 0):
                x_1 = x_0
                x_2 = x_0 + 0.5
            else:
                errorMess = 'x_0 cannot be zero!'
    else:
        if (func == GTT.GhostCellMaterialStencil):
            
            polyInterp = out
            
            omega, order, matInd, centCellInd, offDiagInd = args
            
            # Create vector of cell bounds.
            bounds = GTT.MaterialInterpBounds(omega, order, matInd, offDiagInd)
            
            xVec, x_1, x_2 = GTT.MaterialInterpVec(omega, order, centCellInd, offDiagInd)
            
            
        else:
            errorMess = 'Something\'s up here.'
    
    if (errorMess != ''):
        sys.exit(errorLoc + errorMess)
    
    # Find intervals of cell bounds.
    h = bounds[:-1] - bounds[1:]
    
    delX = x_2 - x_1
    print(polyInterp)
    
    # Iterate through monomials up to appropriate order of accuracy to test stencil.
    for k in range(order + 2):
        coefs = np.zeros(k + 1, float)
        coefs[0] = const
        p = np.poly1d(coefs)
        P = np.polyint(p)
        print('Order ' + str(k) + ':')
        print('p(x) =\n', p)
        print('P(x) =\n', P)
        v = (P(bounds[:-1]) - P(bounds[1:])) / h

        theor = (P(x_2) - P(x_1)) / delX
        act = v.transpose() @ polyInterp
        error = act - theor
        print(theor, act)
        print('Error = ' + str(error))
        print('')
#         if (k < order + 1):
#             assert(np.isclose(act, theor, rtol = 0, atol = tol))
    return
    

4


In [7]:
print(order)
args1 = [order, 0.5]
print(*args1)
TestPoly(GTT.GhostCellStencil, args1)

4
4 0.5
[ 0.08571429 -0.41428571  0.78571429  0.57142857 -0.02857143]
Order 0:
p(x) =
  
2
P(x) =
  
2 x
2.0 1.9999999999999998
Error = -2.220446049250313e-16

Order 1:
p(x) =
  
2 x
P(x) =
    2
1 x
0.5 0.5000000000000002
Error = 2.220446049250313e-16

Order 2:
p(x) =
    2
2 x
P(x) =
         3
0.6667 x
0.16666666666666666 0.16666666666666652
Error = -1.3877787807814457e-16

Order 3:
p(x) =
    3
2 x
P(x) =
      4
0.5 x
0.0625 0.06250000000000011
Error = 1.1102230246251565e-16

Order 4:
p(x) =
    4
2 x
P(x) =
      5
0.4 x
0.025 0.025000000000000078
Error = 7.632783294297951e-17

Order 5:
p(x) =
    5
2 x
P(x) =
         6
0.3333 x
0.010416666666666666 -0.7395833333333335
Error = -0.7500000000000001



In [8]:
hs = omega.h
    
spots = np.roll(hs, -1) - hs

if (all(spots == 0)):
    p = []
    q = []
    NU = False
else:
    # Index before fine-coarse interface
    p = np.where(spots > 0)[0][0]
    # Index before coarse-fine interface
    q = np.where(spots < 0)[0][0]
    NU = True


offDiagInd = int(order / 2)
matInd = p - 2
centCellInd = int(matInd + 1)

args2 = [omega, order, matInd, centCellInd, offDiagInd]

In [9]:
TestPoly(GTT.GhostCellMaterialStencil, args2)

this one.
Bounds are: [0.40625 0.4375  0.46875 0.5     0.5625  0.625  ] <= 0.40625
this one.
Bounds are: [0.40625 0.4375  0.46875 0.5     0.5625  0.625  ] <= 0.40625
[ 0.08571429 -0.41428571  0.78571429  0.57142857 -0.02857143]
Order 0:
p(x) =
  
2
P(x) =
  
2 x
2.0 1.999999999998181
Error = -1.8189894035458565e-12

Order 1:
p(x) =
  
2 x
P(x) =
    2
1 x
1.03125 1.0312499999983231
Error = -1.6768808563938364e-12

Order 2:
p(x) =
    2
2 x
P(x) =
         3
0.6667 x
0.5319010416666665 0.5319010416654404
Error = -1.2261303083960229e-12

Order 3:
p(x) =
    3
2 x
P(x) =
      4
0.5 x
0.2744293212890625 0.2744293212882478
Error = -8.146816554699399e-13

Order 4:
p(x) =
    4
2 x
P(x) =
      5
0.4 x
0.14163246154785158 0.14163246154733686
Error = -5.147271497918382e-13

Order 5:
p(x) =
    5
2 x
P(x) =
         6
0.3333 x
0.07311853766441345 0.07311782240836029
Error = -7.15256053163138e-07

